# LangChain Rag Tutorial
* https://python.langchain.com/docs/tutorials/rag/#:~:text=These%20applications%20use%20a%20technique%20known

# Concepts
* **Indexing**
* **Retrieval and generation**

## Indexing
1. Load: use **Document Loaders** to load documents from various sources like CSV, JSON, PDF, etc.
2. Split: **Text Spliters** split the documents into smaller chunks for indexing.
3. Store: **VectorStore** and **Embeddings** store the indexed documents in a searchable format.

## Retrieval and Generation
4. Retrieve: **Retriever** finds relevant documents from the vector store based on user queries.
5. Generate: **ChatModel**/**LLM** generates answers based on the retrieved documents.